In [1]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["ing_industrial.pdf"]).load_data()

Define LLM and Embedding model

In [5]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [6]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

Define Summary Index and Vector Index over the Same Data

In [7]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

Define Query Engines and Set Metadata

In [8]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [9]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to the careers"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the careers paper."
    ),
)

Define Router Query Engine

In [10]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [11]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: This choice indicates that the document is useful for summarization questions related to careers..
The document provides information about the Bachelor's Degree in Industrial Engineering offered by Utel. It covers the program's objectives, areas of study, career opportunities, academic validation, equivalency in the United States, areas of concentration, total credits, subjects, available experiences, and the quality of education provided by Utel. The degree focuses on preparing students to design processes, influence all areas of a company, and optimize production processes while considering environmental impact. It also highlights the various career paths graduates can pursue, the academic recognition of the program, and the global community and support available to students.


In [12]:
print(len(response.source_nodes))

3


In [13]:
response = query_engine.query(
    "What i´m going to learn in the Industrial Engineering Degree"
)
print(str(response))

Selecting query engine 0: The Industrial Engineering Degree is related to careers and involves summarizing information about various aspects of the field..
You will learn about strategic business planning, business process alignment, control systems development, process management, quality control, manufacturing processes, logistics, and waste reduction. Additionally, you will gain knowledge in equipment, energy, materials, and high-quality production processes with a focus on environmental considerations.


Let's put everything together

In [15]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("ing_industrial.pdf")

In [16]:
response = query_engine.query("Were can i work?")
print(str(response))

Selecting query engine 1: The question is asking for specific information about where to work, which aligns more closely with retrieving specific context from the MetaGPT paper..
You can work in various fields such as administration, information technology, finance, and industrial processes after completing the Bachelor of Science in Industrial Engineering program from Utel.
